In [1]:
import sys
sys.path.append('..')
from src import datagen
from src import engine
import yaml
from time import time
import os

In [2]:
def pid_convert(pid):
    '''
    Converts pids that in the form of integers to strings in the proper format
    
    Example: 012368 --> 001_S_2368
    
             12345  --> 001_S_2345
    '''
    p = pid.item()
    if 100000 < p < 1000000:
        p = '0'+str(p)[:2] + '_S_' + str(p)[2:]
    elif p < 100000 :
        p = '00' + str(p)[:1] + '_S_' + str(p)[1:]
    else:
        p = str(p)[:3] + '_S_' + str(p)[3:]
    return p

In [3]:
with open('../data/datagen_val.pickle','rb') as f:
    dgv = pickle.load(f)

In [ ]:
def save_pids(val_loader,path = '../data/pids.pickle'):
    '''
    Input is a DataLoader object for a specific value of T where T is the number of
    visits used for prediction
    '''
    pids = []
    for x,y in val_loader:
        #print(x.keys())
        #print('y : ', y)
        #print('tau : ', x['tau'])
        #print('pid : ', x['pid'])
        #print('traj_id : ', x['trajectory_id'])
        #print('flag_ad : ', x['flag_ad'])
        #print('first_occurance_ad : ', x['first_occurance_ad'])
        #
        for p in x['pid']:
            p = pid_convert(p)
            pids.append(p)
        #pids.append(x['pid'])
        
    pids = np.unique(pids)
    print(pids)
    
    with open(path,'wb') as f:
        pickle.dump(pids,f)



In [8]:
class Trajectory_Stats:
    def __init__(self, pid, y, y_pred, trajectory_id):
        self.pid = pid
        self.y = y
        self.y_pred = y_pred
        self.trajectory_id = traj_id

class Trajectories_Stats
    def __init__(self,pid,trajectories)
        

In [ ]:
with open('../data/pids.pickle','rb') as f:
    unique_pids = pickle.load(f)

In [4]:
config_file = '../configs/config_baseline.yaml'
with open(config_file) as f:
    config = yaml.load(f)

model = engine.Engine(config['model'])
output = model.test_stats(dgv)


/home/joie/anaconda3/envs/flare_py3/lib/python3.6/site-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until
../src/engine.py:24: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  model_dict = yaml.load(f)
873it [00:08, 99.71it/s] 
538it [00:05, 94.81it/s] 
293it [00:03, 92.85it/s] 
128it [00:01, 86.19it/s]
35it [00:00, 74.70it/s]


In [25]:
with open('../data/stats.pickle','rb') as f:
    output = pickle.load(f)
print(len(output[0]['y']))

3489


In [26]:
def gen_patient_dict(val_dict_T):
    patients = {}
    num_entries = len(val_dict_T['pid'])
    with open('../data/pids.pickle','rb') as f:
        unique_pids = pickle.load(f)
    patients = {}
    
    # initialize patient dict
    for x in unique_pids:
        patients[x] = []
    
    print(val_dict_T)
    for i in range(num_entries):
        pid = pid_convert(val_dict_T['pid'][i])
        temp_traj = Trajectory_Stats(pid, val_dict_T['y'][i],val_dict_T['y_pred'][i],val_dict_T['trajectory_id'][i])
        patients[pid].append(temp_traj)
    return patients

        

In [27]:
patients = gen_patient_dict(output[0])

{'y': tensor([1., 1., 2.,  ..., 1., 1., 0.]), 'y_pred': tensor([[0.0517, 0.9391, 0.0092],
        [0.0096, 0.9900, 0.0004],
        [0.0191, 0.5412, 0.4397],
        ...,
        [0.0063, 0.9896, 0.0041],
        [0.0158, 0.9837, 0.0004],
        [0.9981, 0.0019, 0.0001]], requires_grad=True), 'tau': tensor([1, 2, 3,  ..., 5, 1, 1]), 'pid': tensor([ 820928,  414877, 1374258,  ..., 1094380,  724445,  244158]), 'trajectory_id': tensor([[3, 4],
        [2, 4],
        [0, 3],
        ...,
        [0, 5],
        [0, 1],
        [0, 1]])}


In [28]:
print(patients['007_S_4387'][2].y_pred)

tensor([0.9991, 0.0009, 0.0000], grad_fn=<SelectBackward>)


In [18]:
for key in patients.keys():
    traj_list = patients[key]
    traj_list.sort(key=lambda traj: traj.trajectory_id, reverse=True)
    

RuntimeError: bool value of Tensor with more than one value is ambiguous